# Dataset Acquisition and Processing 
The publicly accessible dataset we have chosen to use is [OpenFlights](https://openflights.org/data.html). Due to necessity of reformatting and existence of dirty data, we will implement data cleaning for pre-processing.

## Data Format
We plan to use airports data and routes data provided by OpenFlights. After pre-processing, only subsets will be considered for selected data and their information are shown in below two tables.
- airports data

| Key | Description |
|---|---|
| Airport ID | Unique OpenFlights identifier for this airport. |
| Name | Name of airport. May or may not contain the City name. |
| City | Main city served by airport. May be spelled differently from Name. |
| Country | Country or territory where airport is located. See Countries to cross-reference to ISO 3166-1 codes. |
| Latitude | Decimal degrees, usually to six significant digits. Negative is South, positive is North. |
| Longitude | Decimal degrees, usually to six significant digits. Negative is West, positive is East. |

In [1]:
import numpy as np
import pandas as pd

In [2]:
col_names = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source']
airports = pd.read_csv('airports.dat', names = col_names, header = None)
airports = airports.drop(columns = ['IATA', 'ICAO', 'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source'])

- routes data

| Key | Description |
|---|---|
| Airline | 2-letter (IATA) or 3-letter (ICAO) code of the airline. |
| Airline ID | Unique OpenFlights identifier for airline (see Airline). |
| Source airport | 3-letter (IATA) or 4-letter (ICAO) code of the source airport. |
| Source airport ID | Unique OpenFlights identifier for source airport (see Airport) |
| Destination airport | 3-letter (IATA) or 4-letter (ICAO) code of the destination airport. |
| Destination airport ID | Unique OpenFlights identifier for destination airport (see Airport) |
| Stops | Number of stops on this flight ("0" for direct) |

In [3]:
col_names = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']
routes = pd.read_csv('routes.dat', names = col_names, header = None)
routes = routes.drop(columns = ['Codeshare', 'Equipment'])

## Data Correction
1. Transfer the raw data into dataframe and use [Pandas](https://pandas.pydata.org/) package to implement the data wrangling. In Pandas missing data is represented by NaN. Drop entries contaning NaN.
2. From the primary key, airport ID, join two data tables. After natural join of two tables, there will not be any missing data.
3. Finally, export the corrected data into a new file.


In [4]:
# clean NaN
routes = routes[routes['Airline ID'].apply(lambda x: str(x).isdigit())]
routes = routes[routes['Source airport ID'].apply(lambda x: str(x).isdigit())]
routes = routes[routes['Destination airport ID'].apply(lambda x: str(x).isdigit())]

# data type: object -> string -> int
routes['Airline ID'] = routes['Airline ID'].astype(str).astype(int)
routes['Source airport ID'] = routes['Source airport ID'].astype(str).astype(int)
routes['Destination airport ID'] = routes['Destination airport ID'].astype(str).astype(int)

# join tables
routes = routes.merge(airports, left_on='Source airport ID', right_on='Airport ID')
routes = routes.merge(airports, left_on='Destination airport ID', right_on='Airport ID', suffixes=('_1', '_2'))

## Distance Between Two Points on Earth
The dataset contains longitude and latitude data of each airport. With [haversine formula](https://en.wikipedia.org/wiki/Haversine_formula), we can compute [great-circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) (the distance between points on the surface of the earth) between each airport.

In [5]:
def distance(routes):
    # The math module contains a function named radians which converts from degrees to radians.
    lon1 = np.radians(routes['Longitude_1'])
    lon2 = np.radians(routes['Longitude_2'])
    lat1 = np.radians(routes['Latitude_1'])
    lat2 = np.radians(routes['Latitude_2'])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of earth. Use 6371 for kilometers. Use 3956 for miles
    r = 6371
    
    # calculate the result
    return(c * r)

In [6]:
routes['distance'] = distance(routes)
routes.to_csv('input_detail.dat', index = False, header = False)
display(routes)

routes = routes.drop(columns = ['Airline', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 'Stops', 'Name_1', 'City_1', 'Country_1', 'Latitude_1', 'Longitude_1', 'Name_2', 'City_2', 'Country_2', 'Latitude_2', 'Longitude_2'])
routes.to_csv('input.dat', index = False, header = False)
display(routes)

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Stops,Airport ID_1,Name_1,City_1,Country_1,Latitude_1,Longitude_1,Airport ID_2,Name_2,City_2,Country_2,Latitude_2,Longitude_2,distance
0,2B,410,AER,2965,KZN,2990,0,2965,Sochi International Airport,Sochi,Russia,43.449902,39.956600,2990,Kazan International Airport,Kazan,Russia,55.606201,49.278702,1506.825604
1,2B,410,ASF,2966,KZN,2990,0,2966,Astrakhan Airport,Astrakhan,Russia,46.283298,48.006302,2990,Kazan International Airport,Kazan,Russia,55.606201,49.278702,1040.438320
2,2B,410,CEK,2968,KZN,2990,0,2968,Chelyabinsk Balandino Airport,Chelyabinsk,Russia,55.305801,61.503300,2990,Kazan International Airport,Kazan,Russia,55.606201,49.278702,770.508500
3,2B,410,DME,4029,KZN,2990,0,4029,Domodedovo International Airport,Moscow,Russia,55.408798,37.906300,2990,Kazan International Airport,Kazan,Russia,55.606201,49.278702,715.649350
4,S7,4329,DME,4029,KZN,2990,0,4029,Domodedovo International Airport,Moscow,Russia,55.408798,37.906300,2990,Kazan International Airport,Kazan,Russia,55.606201,49.278702,715.649350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66311,WJ,5399,YRG,5461,YMN,5492,0,5461,Rigolet Airport,Rigolet,Canada,54.179699,-58.457500,5492,Makkovik Airport,Makkovik,Canada,55.076900,-59.186401,110.244672
66312,WJ,5399,YSO,7252,YMN,5492,0,7252,Postville Airport,Postville,Canada,54.910500,-59.785070,5492,Makkovik Airport,Makkovik,Canada,55.076900,-59.186401,42.434553
66313,WJ,5399,YSO,7252,YHO,5502,0,7252,Postville Airport,Postville,Canada,54.910500,-59.785070,5502,Hopedale Airport,Hopedale,Canada,55.448299,-60.228600,66.099267
66314,ZL,4178,JCK,6276,RCM,9904,0,6276,Julia Creek Airport,Julia Creek,Australia,-20.668301,141.723007,9904,Richmond Airport,Richmond,Australia,-20.701900,143.115005,144.852937


,Airline ID,Airport ID_1,Airport ID_2,distance
0,410,2965,2990,1506.825604
1,410,2966,2990,1040.438320
2,410,2968,2990,770.508500
3,410,4029,2990,715.649350
4,4329,4029,2990,715.649350
...,...,...,...,...
66311,5399,5461,5492,110.244672
66312,5399,7252,5492,42.434553
66313,5399,7252,5502,66.099267
66314,4178,6276,9904,144.852937
